In [13]:
# install necessary packages
!pip install -r biobert_re/requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [ ]:
import tensorflow as tf
import numpy as np
import copy
from bert import modeling,optimization,tokenization
from biobert_re.bert_predictor import BertPredictor
from biobert_re.data.Input import InputExample, InputFeatures, example2feature

# Configuration

In [2]:
config_path = 'biobert/biobert_base/bert_config.json'
checkpoint_path = 'biobert/re_outputs_1/model.ckpt-449'
dict_path = 'biobert/biobert_base/vocab.txt'

max_seq_length = 128
bert_config = modeling.BertConfig.from_json_file(config_path)

In [ ]:
tokenizer = tokenization.FullTokenizer(vocab_file=dict_path,do_lower_case=False)

# Convert sample to feature

In [5]:
guid = 0
sentence = "In comparison, @GENE$ is distant from and less related to the @DISEASE$. "

In [6]:
example = InputExample(guid = guid, text = sentence)
feature = example2feature(example,tokenizer,128)

# convert to numpy array
input_ids = np.reshape(np.array(feature.input_ids),(1,-1))
input_mask = np.reshape(np.array(feature.input_mask),(1,-1))

tokens: ['In', 'comparison', ',', '@', 'GE', '##NE', '$', 'is', 'distant', 'from', 'and', 'less', 'related', 'to', 'the', '@', 'D', '##IS', '##EA', '##SE', '$', '.']


# Build computation graph and initialize weigths from checkpoint


In [7]:
# construct graph
model = BertPredictor(bert_config)
# initialize weights
model.initialize_ckpt(checkpoint_path)
# get probability tensor
probabilities = model.predict()

# Start Session and get prediction

In [8]:
# run session to get probability
with tf.Session() as sess:
    # sess.run(tf.variables_initializer(tf.global_variables()))
    sess.run(tf.global_variables_initializer())
    # saver.restore(sess,save_path=checkpoint_path)
    probability = sess.run(probabilities,feed_dict={
        model.input_ids: input_ids,
        model.input_mask: input_mask
    })

In [9]:
probability

array([[0.9280718 , 0.07192817]], dtype=float32)